In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
main_2020 = pd.read_csv('main_2020_all.csv')
print(main_2020.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18408 entries, 0 to 18407
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   year                            18408 non-null  int64  
 1   quarter                         18408 non-null  object 
 2   monthsigned                     18408 non-null  float64
 3   market                          18408 non-null  object 
 4   region                          18408 non-null  object 
 5   city                            18408 non-null  object 
 6   state                           18408 non-null  object 
 7   internal_submarket              18408 non-null  object 
 8   internal_class                  18408 non-null  object 
 9   leasedSF                        18408 non-null  float64
 10  internal_industry               18408 non-null  object 
 11  transaction_type                18408 non-null  object 
 12  space_type                      

In [4]:
main_2020['quarter_encoded'] = main_2020['quarter'].replace({
    'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4
})

/tmp/ipykernel_3624/3549346842.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  main_2020['quarter_encoded'] = main_2020['quarter'].replace({


In [5]:
us_cities = pd.read_csv("uscities.csv")

main_2020_cy = main_2020
main_2020_cy = main_2020.merge(
    us_cities[['city', 'state_id', 'county_name']],   # pick the columns you need
    left_on=['city', 'state'],                        # columns in main_2020
    right_on=['city', 'state_id'],                    # columns in us_cities
    how='left'
)

# Optionally, rename 'county_name' to just 'county'
main_2020_cy.rename(columns={'county_name': 'county'}, inplace=True)

# Check results
# print(main_2020_cy[['city', 'state', 'county']].drop_duplicates().head(10))
main_2020_cy = main_2020_cy[main_2020_cy['county'].isin(main_2020_cy['county'].value_counts()[lambda x: x >= 3].index)]
main_2020_cy.info()


<class 'pandas.core.frame.DataFrame'>
Index: 17512 entries, 0 to 18407
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   year                            17512 non-null  int64  
 1   quarter                         17512 non-null  object 
 2   monthsigned                     17512 non-null  float64
 3   market                          17512 non-null  object 
 4   region                          17512 non-null  object 
 5   city                            17512 non-null  object 
 6   state                           17512 non-null  object 
 7   internal_submarket              17512 non-null  object 
 8   internal_class                  17512 non-null  object 
 9   leasedSF                        17512 non-null  float64
 10  internal_industry               17512 non-null  object 
 11  transaction_type                17512 non-null  object 
 12  space_type                      17512

In [6]:
# ----- Step 1: Define Rent Range Bins using "direct_internal_class_rent" -----
# Adjust these bin edges as appropriate for your dataset.
rent_bins = [0, 50, 100, 200, 1000]  
rent_labels = ['Low', 'Medium', 'High', 'Very High']
# Here we use "direct_internal_class_rent" to bin the data.
main_2020_cy['rent_range'] = pd.cut(main_2020_cy['direct_internal_class_rent'], bins=rent_bins, labels=rent_labels, right=False)

# ----- Step 2: Define the industries of interest -----
industries = ["Tech", "Financial", "Legal"]

# Dictionary to hold the score DataFrames for each combination.
score_lists = {}

# ----- Step 3: Loop Over Industries and Rent Ranges -----
for industry in industries:
    # Filter for the given industry (assuming "internal_industry" holds these values).
    df_ind = main_2020_cy[main_2020_cy['internal_industry'] == industry].copy()
    
    for rent_range in rent_labels:
        # Filter further by the rent range.
        df_subset = df_ind[df_ind['rent_range'] == rent_range]
        if df_subset.empty:
            continue  # Skip if no data in this combination.
        
        # Compute county-level cluster counts.
        county_cluster_counts = df_subset.groupby('county')['cluster'].value_counts().unstack(fill_value=0)
        
        # Convert counts to percentages.
        county_cluster_pct = county_cluster_counts.div(county_cluster_counts.sum(axis=1), axis=0) * 100
        
        # ----- Step 4: Define Cluster Weights and Compute Scores -----
        # Here, assume n_clusters=6 and clusters 0 to 5 are present.
        cluster_weights = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6}
        
        def compute_county_score(row, weights):
            return sum(row[cluster] * weights.get(cluster, 0) for cluster in row.index)
        
        county_scores = county_cluster_pct.apply(lambda row: compute_county_score(row, cluster_weights), axis=1)
        county_scores = county_scores.reset_index(name='county_value_score')
        
        # Optionally, sort the scores descending so higher scores appear first.
        county_scores = county_scores.sort_values(by='county_value_score', ascending=False)
        
        # Save the score list for this industry and rent range.
        score_lists[(industry, rent_range)] = county_scores
        
        # Print a header and preview of the results.
        print(f"\nIndustry: {industry}, Rent Range: {rent_range}")
        print(county_scores.head())
        
        # (Optional) Export each to CSV:
        filename = f"county_scores_{industry}_{rent_range}.csv"
        county_scores.to_csv(filename, index=False)
